In [1]:
import pandas as pd
import numpy as np
from DataProcessing import DataProcessing
from Network import Network
import datetime
import tensorflow as tf
from DataVisualization import DataVisualization

C:\Users\omid\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\omid\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\omid\AppData\Local\conda\conda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\User

In [2]:
dataset = pd.read_csv('PV.csv')
print("The shape of raw data is ", dataset.shape)
print(dataset.head(10))

The shape of raw data is  (8833, 13)


Index(['Index', 'local_timestamp', 'Time_Index', 'Ambient Dew Point (°C)',
       'Ambient Humidity (%)', 'Ambient Temperature (°C)',
       'Barometric Pressure (mb)', 'Rain Accumulation Record (mm)',
       'Rain Accumulation for Day (mm)', 'Rain Intensity (mm/hr)',
       'Rain Intensity Peak (mm/hr)', 'POA Irradiance (W/m²)',
       'Aggregate AC Power (kW)'],
      dtype='object')

In [3]:
dv = DataVisualization(dataset)
dv.show_raw_visualization()
dv.show_heatmap()

      local_timestamp  Aggregate AC Power (kW)
0 2017-05-16 00:15:00                      0.0
1 2017-05-16 00:15:00                      0.0
2 2017-05-16 00:30:00                      0.0
3 2017-05-16 00:45:00                      0.0
4 2017-05-16 01:00:00                      0.0
5 2017-05-16 01:15:00                      0.0
6 2017-05-16 01:30:00                      0.0
7 2017-05-16 01:45:00                      0.0
8 2017-05-16 02:00:00                      0.0
9 2017-05-16 02:15:00                      0.0


F:\PROGRAMMER\python\PythonProjects\StockBroker\DataLoading.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data['local_timestamp'] = pd.to_datetime(cleaned_data['local_timestamp'])
F:\PROGRAMMER\python\PythonProjects\StockBroker\DataLoading.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data.sort_values(['local_timestamp'], ascending=True, inplace=True)


In [4]:
dt = DataProcessing(dataset)
dataset_train, dataset_val, inputs_shape, targets_shape = dt.preparing_dataset()
model = Network(input_shape=inputs_shape, learning_rate=0.001).simpleRNN()


shape of data after data cleaning is  (8833, 2)
      local_timestamp  Aggregate AC Power (kW)
0 2017-05-16 00:15:00                      0.0
1 2017-05-16 00:15:00                      0.0
2 2017-05-16 00:30:00                      0.0
3 2017-05-16 00:45:00                      0.0
4 2017-05-16 01:00:00                      0.0
5 2017-05-16 01:15:00                      0.0
6 2017-05-16 01:30:00                      0.0
7 2017-05-16 01:45:00                      0.0
8 2017-05-16 02:00:00                      0.0
9 2017-05-16 02:15:00                      0.0


In [8]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
path_checkpoint = "model_checkpoint.h5"
es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=100)

modelckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

history = model.fit(
    dataset_train,
    epochs=1000,
    validation_data=dataset_val,
    callbacks=[tensorboard_callback, es_callback, modelckpt_callback],
)

In [9]:
dv.visualize_loss(history, "Training and Validation Loss")

for x, y in dataset_val.take(5):
    dv.show_plot(
        [x[0][:, 0].numpy(), y[0].numpy(), model.predict(x)[0]],
        1,
        "Single Step Prediction")

AttributeError: 'tuple' object has no attribute 'ndims'